<a href="https://colab.research.google.com/github/shyanasri/LSTMBiasClassifier/blob/main/ZeroShot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries

In [2]:
!pip install transformers


Import libraries

In [ ]:
#LSTM for multi-categorical classification
import csv
import tensorflow as tf
import numpy as np
import re
import matplotlib.pyplot as plt
import collections
from keras_self_attention import SeqSelfAttention

huggingface login

In [13]:
from huggingface_hub import notebook_login
notebook_login()

Load model

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[{"type": "function", "function": {"name": "get_current_weather", "description": "Get the current weather", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"}, "format": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "The temperature unit to use. Infer this from the users location."}}, "required": ["location", "format"]}}}] Given a sentence, I want you to act as a bias/hate speech detector. Please identify if the sentence has any bias about these minority groups (mexican', 'lgbtq', 'asian', 'muslim', 'nativeamerican', 'jewish', 'middleeast', 'mental disability', 'chinese', 'latino', 'black', 'physical disability', 'women') and identify if it's using positive or negative stereotyping. As an example, if the sentence was "Women don't do well in positions of power.", your response would be "This sentence has bias about women and it uses negative stereotyping." Here is the sentence

Prompt model

In [40]:
import csv
from copy import deepcopy
conversation = [{"role": "user",
              "content": "Given a sentence, I want you to act as a bias/hate speech detector. Please identify if the sentence has any bias about these 13 minority groups I've listed ('lgbtq', 'asian', 'nativeamerican', 'jewish', 'middleeast', 'mental disability', 'latino', 'black', 'physical disability', 'women') and identify if it's using positive, neutral, or negative stereotyping. As an example, if the sentence was \"Women don't do well in positions of power.\", your response would be \"This sentence has bias about women and it uses negative stereotyping.\" Here is the sentence I want you to analyze: \"{}\""}]


conversations = []
raw_data = []

with open('failed_texts.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      new_item = []
      new_item = deepcopy(conversation)
      new_item[0]["content"] =  new_item[0]["content"].format(row[0])
      conversations.append(new_item)

label_convert = ['women',
              'lgbtq',
              'mental disability',
              'black',
              'african american'
              'asian',
              'chinese',
              'muslim',
              'native american',
              'middle east',
              'physical disability',
              'latino',
              'jewish']


with open('pred_texts.csv','w', newline='') as csvfile:
  writer = csv.writer(csvfile)

  for i, convo in enumerate(conversations):
  # render the tool use prompt as a string:
    print(convo)
    tool_use_prompt = tokenizer.apply_chat_template(
                convo,
                tokenize=False,
                add_generation_prompt=True,
    )

    inputs = tokenizer(tool_use_prompt, return_tensors="pt")

    outputs = model.generate(**inputs, max_new_tokens=1000)
    op = tokenizer.decode(outputs[0], skip_special_tokens=True)
    op =  op.replace(convo[0]["content"], '')
    print(op)
    cat = ''
    for label in label_convert:
      if label in op.lower():
        cat = label
        break
    op = [op, cat]
    writer.writerow(op)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[{'role': 'user', 'content': 'Given a sentence, I want you to act as a bias/hate speech detector. Please identify if the sentence has any bias about these 13 minority groups I\'ve listed (\'lgbtq\', \'asian\', \'nativeamerican\', \'jewish\', \'middleeast\', \'mental disability\', \'latino\', \'black\', \'physical disability\', \'women\') and identify if it\'s using positive, neutral, or negative stereotyping. As an example, if the sentence was "Women don\'t do well in positions of power.", your response would be "This sentence has bias about women and it uses negative stereotyping." Here is the sentence I want you to analyze: "this statement indicates that black men are seen as savages or animals compared to white men"'}], [{'role': 'user', 'content': 'Given a sentence, I want you to act as a bias/hate speech detector. Please identify if the sentence has any bias about these 13 minority groups I\'ve listed (\'lgbtq\', \'asian\', \'nativeamerican\', \'jewish\', \'middleeast\', \'mental

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 This sentence has bias towards the black community and it uses negative stereotyping. The statement suggests a harmful and derogatory comparison between black men and white men, perpetuating a racist stereotype.
[{'role': 'user', 'content': 'Given a sentence, I want you to act as a bias/hate speech detector. Please identify if the sentence has any bias about these 13 minority groups I\'ve listed (\'lgbtq\', \'asian\', \'nativeamerican\', \'jewish\', \'middleeast\', \'mental disability\', \'latino\', \'black\', \'physical disability\', \'women\') and identify if it\'s using positive, neutral, or negative stereotyping. As an example, if the sentence was "Women don\'t do well in positions of power.", your response would be "This sentence has bias about women and it uses negative stereotyping." Here is the sentence I want you to analyze: " how to be a man\n- how to get over your wife leaving you for another woman"'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 This sentence does not explicitly mention any of the listed minority groups. However, it could potentially be interpreted as implying a negative stereotype about people with mental or physical disabilities, as "sick" could be used as a euphemism for those with such conditions. The sentence itself is neutral in terms of bias, but the potential interpretation could be seen as negative stereotyping. It's important to note that the original sentence does not explicitly state that the "sick" are a burden, but the connotation could be inferred. To avoid potential misunderstandings, it would be more appropriate to use terms like "individuals requiring care" or "people with health conditions" instead.
[{'role': 'user', 'content': 'Given a sentence, I want you to act as a bias/hate speech detector. Please identify if the sentence has any bias about these 13 minority groups I\'ve listed (\'lgbtq\', \'asian\', \'nativeamerican\', \'jewish\', \'middleeast\', \'mental disability\', \'latino\', \'b

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 This sentence contains bias towards the racial group "black" and uses negative stereotyping. The use of racial slurs and derogatory language is not acceptable and promotes harmful stereotypes. It's important to promote respect and understanding among all racial and ethnic groups.
[{'role': 'user', 'content': 'Given a sentence, I want you to act as a bias/hate speech detector. Please identify if the sentence has any bias about these 13 minority groups I\'ve listed (\'lgbtq\', \'asian\', \'nativeamerican\', \'jewish\', \'middleeast\', \'mental disability\', \'latino\', \'black\', \'physical disability\', \'women\') and identify if it\'s using positive, neutral, or negative stereotyping. As an example, if the sentence was "Women don\'t do well in positions of power.", your response would be "This sentence has bias about women and it uses negative stereotyping." Here is the sentence I want you to analyze: "many iraqis eat cockroaches for breakfast and scorpions for lunch"'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 This sentence does not explicitly show bias towards the Middle Eastern group (Iraqis). It is a factual statement about a cultural practice in some parts of Iraq. However, it could be perceived as stereotyping if used in a derogatory or demeaning context, but in the given sentence, it is neutral.
[{'role': 'user', 'content': 'Given a sentence, I want you to act as a bias/hate speech detector. Please identify if the sentence has any bias about these 13 minority groups I\'ve listed (\'lgbtq\', \'asian\', \'nativeamerican\', \'jewish\', \'middleeast\', \'mental disability\', \'latino\', \'black\', \'physical disability\', \'women\') and identify if it\'s using positive, neutral, or negative stereotyping. As an example, if the sentence was "Women don\'t do well in positions of power.", your response would be "This sentence has bias about women and it uses negative stereotyping." Here is the sentence I want you to analyze: "Dj Khaled is retarded"'}]
 This sentence has bias towards people wi